In [13]:
import pandas as pd
import numpy as np
from platform import python_version
import numpy as np
import time
import argparse
#from keras.utils import to_categorical
import os
import tensorflow as tf
import keras
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import Input, Layer, Dropout, Masking
from keras.layers import GRU, Activation
from keras.models import Model, Sequential
from keras.models import model_from_json


In [14]:
print('keras version ' + keras.__version__)
print('python version ' + python_version())

keras version 2.1.6
python version 3.6.4


In [15]:
PATH_TO_WORKING_DIR = 'c:/asi/dss/deepLearningRecsys/item_embedding_session_reco/'
PATH_TO_ORIGINAL_DATA = '././data/'
PATH_TO_PROCESSED_DATA = '././pre-train/'
file_2_tokenize_name = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_tr.txt'  #original item id session train
file_2_tokenize_name_test = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_ts.txt'   #original item id session test
tokenized_file_name = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_tr_tk.txt'  #embedded vector session train
tokenized_file_name_test = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_ts_tk.txt'    #embedded vector session test
tokenized_file_name_X = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_tr_tkX.txt'      #GRU matrix input
tokenized_file_name_Y = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_tr_tkY.txt'      #model actual prediction 
tokenized_file_name_test_X = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_ts_tkX.txt'  #GRU matrix input test
tokenized_file_name_test_Y = PATH_TO_PROCESSED_DATA + 'rcs15_sesss_ts_tkY.txt'
glove_vectors_file_name = PATH_TO_PROCESSED_DATA + 'vectors_s_rcs15.txt'        #GloVe vectors
glove_vocab_file_name = PATH_TO_PROCESSED_DATA + 'rcs15_s_vocab.txt'            #Glove vocab
vocab_size = 14386
sessions_train = 781486
sessions_test = 30726 
sequence_length = 3
sequence_len = 3
embedding_size = 50
do_val = 0.2
n_samples = 10
hidden_neurons = 30  #RNN layer output dim
num_iter = 20
N = 10  #recall@
maxW = N+3   #max item clicke we count

In [16]:
def to_categorical(y, nb_classes=None):
    '''Convert class vector (integers from 0 to nb_classes)
    to binary class matrix, for use with categorical_crossentropy.
    '''
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes),dtype=np.bool)
    for i in range(len(y)):
        idx = y[i,0]
        Y[i, idx] = 1
    return Y


In [17]:
def calcRecall(recom,actual,i):
    act_set = set(actual)
    pred_set = set(recom[:i])
    result = (float) (len(act_set & pred_set)) / float(len(act_set))
    return result

In [18]:
#generate GloVe vocabulary and vectors dictionary
def generate(glove_vectors_file_name,glove_vocab_file_name):
    with open(glove_vocab_file_name, 'r') as f:
        words = [x.rstrip().split(' ')[0] for x in f.readlines()]
    with open(glove_vectors_file_name, 'r') as f:
        vectors = {}
        for line in f:
            vals = line.rstrip().split(' ')
            vectors[vals[0]] = [float(x) for x in vals[1:]]

    vocab_size = len(words)
    vocab = {w: idx for idx, w in enumerate(words)}
    ivocab = {idx: w for idx, w in enumerate(words)}

    vector_dim = len(vectors[ivocab[0]])
    W = np.zeros((vocab_size, vector_dim))
    for word, v in vectors.items():
        if word == '<unk>':
            continue
        W[vocab[word], :] = v

    # normalize each word vector to unit variance
    W_norm = np.zeros(W.shape)
    d = (np.sum(W ** 2, 1) ** (0.5))
    W_norm = (W.T / d).T
    return (W_norm, vocab, ivocab)

In [19]:
#load voacab and vectors file to dict
os.chdir(PATH_TO_WORKING_DIR)
W, vocab, ivocab = generate(glove_vectors_file_name, glove_vocab_file_name)

In [20]:
#load test files
os.chdir(PATH_TO_WORKING_DIR)
print("before load test")

#load first 3 clicks embedded vectors
XTS = np.loadtxt(tokenized_file_name_test_X).reshape((sessions_test,sequence_len,embedding_size))
test_size = int(XTS.shape[0])
X_test = XTS[0:test_size, :]

#load next user clicks id
with open(file_2_tokenize_name_test, 'r') as f:
    full_data = [line.rstrip().split(' ') for line in f]
    data = full_data

print("after load test")

before load test
after load test


In [21]:
model = Sequential()

#load and create model
json_string = 'm3_10_my_model_architecture.json'
json_file = open(PATH_TO_PROCESSED_DATA + json_string, 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

print(model.summary())

# load weights into new model
weights_string = 'm3_10_my_model_weights.h5'
model.load_weights(PATH_TO_PROCESSED_DATA +weights_string)
print("Loaded model from disk")
 
# evaluate loaded model on test data
model.compile(optimizer='adam',loss='categorical_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 30)                7290      
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 14260)             442060    
_________________________________________________________________
activation_1 (Activation)    (None, 14260)             0         
Total params: 449,350
Trainable params: 449,350
Non-trainable params: 0
_________________________________________________________________
None
Loaded model from disk


In [22]:
#use first 3 clicks to recommend
Y_test = model.predict(X_test)
recall = 0
rown = 0
rownc = 0
recallV = [0]*N
print(len(data))

print("testing model ")
for row in data:
    if len(row) < 4:  #nothing to predict
        rown = rown + 1
        continue  #nothin to predict
    
    predicted_results = Y_test[rown]
    dist = predicted_results
    input_term = row[0:3]
    for term in input_term: 
         if term in vocab:
            index = vocab[term]
            dist[index] = -np.Inf
    top = np.argsort(-dist)[:N]
    top_org = []
    for glv_item_id in top:
        org_item_id =ivocab[glv_item_id]
        top_org.append(org_item_id)
    top_org_set = set(top_org)
    match = [click_id for click_id in row[3:maxW] if click_id in top_org_set]
    recall = recall + (float (len(match))/ (float) (len(row[3:maxW])))
    rownc = rownc+1
    rown = rown + 1
    for i in range(N):
        recallV[i]= recallV[i] + calcRecall(top_org,row[3:maxW],(i+1))

recall = (float) (recall)/(float) (rownc)
print('recall ',recall)

for i in range(N):
    recallV[i] = (float)(recallV[i])/(float)(rownc)
    print('recall@ '+ str(i+1) + ' ' + str(recallV[i])+"\n")
              



30726
testing model 
recall  0.2975492938610506
recall@ 1 0.08354878087840382

recall@ 2 0.13602873942651383

recall@ 3 0.1725829890202079

recall@ 4 0.20154218959383785

recall@ 5 0.2243835296925494

recall@ 6 0.24426899456434495

recall@ 7 0.2610553442666291

recall@ 8 0.2742515645195771

recall@ 9 0.28630451280080205

recall@ 10 0.29717016197439955



In [ ]:
#predict clicked items for one session
sess_ind = 0  @session index to predict
prediction = Y_test[sess_ind]
top = np.argsort(-prediction)[:N]

<p style="color:DarkMagenta;">**???** convert top predictions from GloVe index to original index and print </p>

In [ ]:
click_ids = data[sess_ind]
#first 3 clicks for the session
click_ids[:???]
#following clicks 
click_ids[???:]

<p style="color:DarkMagenta;">**???** print clicked ids and find matches </p>